In [8]:
import sys
import os
import numpy as np
from models.convnet import ConvNet1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils

In [9]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 200

In [10]:
BASE_DIR = './data/'
GLOVE_DIR = BASE_DIR + '/glove.6B/'
model_path = './models/cnn1d.h5'

In [11]:
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [6]:
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [14]:
# prepare embedding matrix
print('Preparing the Embedding Matrix')
nb_words = len(word_index)
embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Preparing the Embedding Matrix


NameError: name 'word_index' is not defined

In [7]:
text = "French presidential candidate Emmanuel Macrons anti-system angle is a sham | Philippe Marlire"
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)

NameError: name 'Tokenizer' is not defined

In [6]:
print text
print sequences

15 Horrible 90s Throwbacks. You Will Be Glad These Are In The Past. I Know I Am.
[[14], [16], [], [4], [6], [8], [8], [3], [10], [5], [1], [], [17], [15], [7], [], [9], [4], [8], [6], [11], [10], [2], [18], [12], [7], [], [], [24], [6], [23], [], [11], [3], [5], [5], [], [10], [1], [], [20], [5], [2], [19], [], [9], [4], [1], [7], [1], [], [2], [8], [1], [], [3], [13], [], [9], [4], [1], [], [22], [2], [7], [9], [], [], [3], [], [12], [13], [6], [11], [], [3], [], [2], [21], []]


In [7]:
word_index = tokenizer.word_index

In [22]:
word_index

{"'": 14,
 'a': 5,
 'c': 13,
 'e': 2,
 'g': 8,
 'h': 15,
 'i': 3,
 'j': 16,
 'l': 6,
 'm': 11,
 'n': 7,
 'o': 9,
 'p': 17,
 'r': 10,
 's': 1,
 't': 4,
 'u': 12,
 'w': 18}

In [8]:
print('Found %s unique tokens.' % len(word_index))

NameError: name 'word_index' is not defined

In [37]:
text = "1".lower()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)
input_sent = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [16]:
model = ConvNet1D(vocab_size=len(embeddings_index), embedding_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)

In [33]:
input_sent

array([[0, 0, 0, ..., 0, 0, 2],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ..., 
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 3],
       [0, 0, 0, ..., 0, 0, 4]], dtype=int32)

In [18]:
from keras.models import load_model
model = load_model('models/cnn1d.h5')

In [38]:
m.predict(input_sent)[0]

array([ 0.38162312,  0.61837685], dtype=float32)

In [2]:
json_string = model.to_json()

In [3]:
from keras.models import model_from_json
m = model_from_json(json_string)

In [4]:
m.load_weights('models/cnn1d_w.h5')

In [5]:
m.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 1000, 200)     8224800     embedding_input_2[0][0]          
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)  (None, 996, 128)      128128      embedding_1[0][0]                
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 996, 128)      0           convolution1d_1[0][0]            
____________________________________________________________________________________________________
maxpooling1d_1 (MaxPooling1D)    (None, 199, 128)      0           activation_1[0][0]               
___________________________________________________________________________________________

In [23]:
import string
import re

MATCH_MULTIPLE_SPACES = re.compile("\ {2,}")
VOCABULARY_SIZE = 6500
UNK = "<UNK>"
PAD = "<PAD>"

def clean(text):
    text = text.lower()
    for punctuation in string.punctuation:
        text = text.replace(punctuation, " " + punctuation + " ")
    for i in range(10):
        text = text.replace(str(i), " " + str(i) + " ")
    text = MATCH_MULTIPLE_SPACES.sub(" ", text)
    return "\n".join(line.strip() for line in text.split("\n"))

In [27]:
cleaner =  clean("Hello Workd!").lower().split()
input_sent = pad_sequences(cleaner, maxlen=MAX_SEQUENCE_LENGTH)

ValueError: invalid literal for long() with base 10: 'hello'